In [1]:
import pandas as pd
import numpy as np
from time import perf_counter

In [2]:
td = pd.DataFrame.from_records(np.random.randn(1000000, 10), columns=list('abcdefghij'))

In [3]:
tic = perf_counter()
td.to_csv('something.csv')
print(perf_counter() - tic)

18.007173412000157


In [4]:
tic = perf_counter()
td.to_msgpack('something.mspk')
print(perf_counter() - tic)

0.2792238729998644


In [5]:
tic = perf_counter()
td.to_hdf('something.h5', '/test')
print(perf_counter() - tic)

0.9678348190000179


In [2]:
import feather

In [7]:
tic = perf_counter()
feather.write_dataframe(td, 'something.feather')
print(perf_counter() - tic)

0.10822407699993164


In [8]:
import fastparquet

In [11]:
tic = perf_counter()
fastparquet.write('something.parq', td, compression='GZIP')
print(perf_counter() - tic)

3.822317546000022


In [31]:
from sqlalchemy import create_engine, Table, MetaData, Column, Float, BigInteger
engine = create_engine('sqlite:///something.db')

In [32]:
# Do this and blow your memory up for 1M records
tic = perf_counter()
td.to_sql('test1', engine, chunksize=10000) 
print(perf_counter() - tic)

17.52586600500217


In [13]:
%load_ext line_profiler

In [33]:
metadata = MetaData()

test_table = Table('test', metadata, 
                   Column('index', BigInteger, index=True),
                   Column('a', Float),
                   Column('b', Float),
                   Column('c', Float),
                   Column('d', Float),
                   Column('e', Float),
                   Column('f', Float),
                   Column('g', Float),
                   Column('h', Float),
                   Column('i', Float),
                   Column('j', Float))

metadata.create_all(engine)
connection = engine.connect()

def chunked_sql(connection, df):
    def chunker(seq, size):
        return (seq[pos:pos + size] for pos in range(0, len(seq), size))

    tic = perf_counter()

    for chunk in chunker(df, 10000):
        params = [c[1].to_dict() for c in chunk.iterrows()]
        connection.execute(test_table.insert(), params)
    print(perf_counter() - tic)

In [34]:
chunked_sql(connection, td)

69.7622901759969


In [20]:
%lprun -f chunked_sql chunked_sql(connection, td)

14.059273199003655


In [23]:
td.values

array([[-1.25677192,  1.31062673, -0.76123246, ...,  0.43792086,
         0.77241304, -1.01673708],
       [ 0.35127832, -0.83995198, -1.00729763, ...,  1.33800725,
         2.28597173,  0.27582778],
       [-1.47702389, -2.62370182, -1.02661425, ...,  2.47913286,
        -2.03562435,  0.53663893],
       ..., 
       [ 0.76223409, -0.24053633,  0.91032359, ...,  0.71536533,
        -0.99861447,  0.01770397],
       [-0.25761616,  0.92855177,  0.52754937, ...,  1.41697664,
         0.69969227,  0.77661552],
       [-1.12170881,  0.16243831, -0.96426112, ...,  1.15192576,
         0.48580664, -0.89289491]])